In [41]:
#Load Libraries
!pip install nltk
!pip install pyldavis
!pip install langdetect
import pandas as pd 
import gensim
from gensim import corpora,models 
from gensim.models import Phrases
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [45]:
# Read data
# use read_csv to read csv file, not read_table
df = pd.read_csv('train.csv')
df

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


In [47]:
#convert all review text into list format
docs = df['Response'].tolist()
docs[1]

'Hello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone.\xa0Changing our feelings is like changing our thoughts - it\'s hard to do. Our minds are so amazing that the minute you change your thought another one can be right there to take it\'s place. Without your permission, another thought can just pop in there. The new thought may feel worse than the last one! My guess is that you have tried several things to improve this on your own even before reaching out on here. People often try thinking positive thoughts, debating with their thoughts, or simply telling themselves that they need to "snap out of it" - which is also a thought that carries some self-criticism.\xa0Some people try a different approach, and there are counselors out there that can help you with this. The idea is that instead of trying to change the t

In [49]:
# Assuming you've loaded your CSV file into a DataFrame `df`
docs = df['Response'].astype(str)  # Replace 'text_column_name' with the name of your column

tokenizer = RegexpTokenizer(r'\w+')

# Now 'docs' is a Series object where each document is a string
# We'll apply the tokenizer to each document in the Series.
docs = docs.str.lower().apply(tokenizer.tokenize)

# Let's take a look at the first 150 tokens of the second document
# Ensure that there are enough tokens for the second document
if len(docs) > 1 and len(docs[1]) > 150:
    print(docs[1][:150])
else:
    print("Document 1 does not have 150 tokens.")

['hello', 'and', 'thank', 'you', 'for', 'your', 'question', 'and', 'seeking', 'advice', 'on', 'this', 'feelings', 'of', 'worthlessness', 'is', 'unfortunately', 'common', 'in', 'fact', 'most', 'people', 'if', 'not', 'all', 'have', 'felt', 'this', 'to', 'some', 'degree', 'at', 'some', 'point', 'in', 'their', 'life', 'you', 'are', 'not', 'alone', 'changing', 'our', 'feelings', 'is', 'like', 'changing', 'our', 'thoughts', 'it', 's', 'hard', 'to', 'do', 'our', 'minds', 'are', 'so', 'amazing', 'that', 'the', 'minute', 'you', 'change', 'your', 'thought', 'another', 'one', 'can', 'be', 'right', 'there', 'to', 'take', 'it', 's', 'place', 'without', 'your', 'permission', 'another', 'thought', 'can', 'just', 'pop', 'in', 'there', 'the', 'new', 'thought', 'may', 'feel', 'worse', 'than', 'the', 'last', 'one', 'my', 'guess', 'is', 'that', 'you', 'have', 'tried', 'several', 'things', 'to', 'improve', 'this', 'on', 'your', 'own', 'even', 'before', 'reaching', 'out', 'on', 'here', 'people', 'often', 't

In [51]:
# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
    
print(docs[1][:50])

['hello', 'and', 'thank', 'you', 'for', 'your', 'question', 'and', 'seeking', 'advice', 'on', 'this', 'feelings', 'of', 'worthlessness', 'is', 'unfortunately', 'common', 'in', 'fact', 'most', 'people', 'if', 'not', 'all', 'have', 'felt', 'this', 'to', 'some', 'degree', 'at', 'some', 'point', 'in', 'their', 'life', 'you', 'are', 'not', 'alone', 'changing', 'our', 'feelings', 'is', 'like', 'changing', 'our', 'thoughts', 'it']


In [53]:
# Remove stopwords.
docs = [[token for token in doc if token not in stopwords.words('english')] for doc in docs]
print(docs[1][:50])

['hello', 'thank', 'question', 'seeking', 'advice', 'feelings', 'worthlessness', 'unfortunately', 'common', 'fact', 'people', 'felt', 'degree', 'point', 'life', 'alone', 'changing', 'feelings', 'like', 'changing', 'thoughts', 'hard', 'minds', 'amazing', 'minute', 'change', 'thought', 'another', 'one', 'right', 'take', 'place', 'without', 'permission', 'another', 'thought', 'pop', 'new', 'thought', 'may', 'feel', 'worse', 'last', 'one', 'guess', 'tried', 'several', 'things', 'improve', 'even']


In [54]:
# Remove stopwords including fashion-specific ones
english_stopwords = set(stopwords.words('english'))
therapy_specific_stopwords = {'u','yes','okay','sure','really','very','quite','is','are','have','be','feel','think','talk','therapy','session','patient','client'}
all_stopwords = english_stopwords.union(therapy_specific_stopwords)
docs = [[token for token in doc if token not in all_stopwords] for doc in docs]
print(docs[1][:50])

['hello', 'thank', 'question', 'seeking', 'advice', 'feelings', 'worthlessness', 'unfortunately', 'common', 'fact', 'people', 'felt', 'degree', 'point', 'life', 'alone', 'changing', 'feelings', 'like', 'changing', 'thoughts', 'hard', 'minds', 'amazing', 'minute', 'change', 'thought', 'another', 'one', 'right', 'take', 'place', 'without', 'permission', 'another', 'thought', 'pop', 'new', 'thought', 'may', 'worse', 'last', 'one', 'guess', 'tried', 'several', 'things', 'improve', 'even', 'reaching']


In [57]:
# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]
print(docs[1][:50])

['hello', 'thank', 'question', 'seeking', 'advice', 'feelings', 'worthlessness', 'unfortunately', 'common', 'fact', 'people', 'felt', 'degree', 'point', 'life', 'alone', 'changing', 'feelings', 'like', 'changing', 'thoughts', 'hard', 'minds', 'amazing', 'minute', 'change', 'thought', 'another', 'one', 'right', 'take', 'place', 'without', 'permission', 'another', 'thought', 'pop', 'new', 'thought', 'may', 'worse', 'last', 'one', 'guess', 'tried', 'several', 'things', 'improve', 'even', 'reaching']


In [59]:
# Lemmatize the documents.

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
print(docs[1][:50])

['hello', 'thank', 'question', 'seeking', 'advice', 'feeling', 'worthlessness', 'unfortunately', 'common', 'fact', 'people', 'felt', 'degree', 'point', 'life', 'alone', 'changing', 'feeling', 'like', 'changing', 'thought', 'hard', 'mind', 'amazing', 'minute', 'change', 'thought', 'another', 'one', 'right', 'take', 'place', 'without', 'permission', 'another', 'thought', 'pop', 'new', 'thought', 'may', 'worse', 'last', 'one', 'guess', 'tried', 'several', 'thing', 'improve', 'even', 'reaching']


In [61]:
# use Phrases to Compute bigrams.


# Add bigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs,min_count = 5, threshold=40)


print(bigram[docs[0]])


['everyone', 'think', 'worthless', 'maybe', 'need', 'find', 'new', 'people', 'hang', 'seriously', 'social', 'context', 'person', 'life', 'big', 'influence', 'self_esteem', 'otherwise', 'go', 'round', 'round', 'trying', 'understand', 'worthless', 'go', 'back', 'crowd', 'knocked', 'many', 'inspirational', 'message', 'find', 'social_medium', 'maybe', 'read', 'one', 'state', 'person', 'worthless', 'everyone', 'good', 'purpose', 'life', 'also', 'since', 'culture', 'saturated', 'belief', 'someone', 'good', 'somehow', 'terrible', 'bad', 'feeling', 'part', 'living', 'motivation', 'remove', 'situation', 'relationship', 'u', 'harm', 'good', 'bad', 'feeling', 'terrible', 'feeling', 'worthlessness', 'may', 'good', 'sense', 'motivating', 'find', 'much', 'better', 'feeling', 'today']


In [ ]:
pyLDAvis.enable_notebook()

# Let's assume 'docs' is your list of preprocessed documents (each document is a list of words)
dictionary = corpora.Dictionary(docs)  # Creates a dictionary of all unique words

# Convert document into the bag-of-words format
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Apply TF-IDF
tfidf = models.TfidfModel(corpus)  # Initialize a model
corpus_tfidf = tfidf[corpus]  # Apply transformation to the entire corpus

# Running LDA using Bag of Words
lda_model_bow = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15, random_state=100)

# Running LDA using TF-IDF
lda_model_tfidf = LdaModel(corpus_tfidf, num_topics=10, id2word=dictionary, passes=15, random_state=100)

# Save the model to disk if needed
lda_model_tfidf.save('model_tfidf.lda')

# View the topics
print(lda_model_tfidf.print_topics())

# Assuming 'lda_model_tfidf' and 'corpus_tfidf' are already defined along with 'dictionary'
vis = pyLDAvis.gensim_models.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
pyLDAvis.display(vis)


In [31]:
pyLDAvis.save_html(vis, 'lda_topic_modeling.html')


### Topic Modeling Visualization (Static Image)

The topic modeling results below were originally created with pyLDAvis as an interactive chart.  
Since GitHub does not support interactive visualizations in notebooks, a static version is shown here.

![LDA Topic Modeling](lda_topic_modeling.png)



In [49]:
# Let's assume 'docs' is your list of preprocessed documents (each document is a list of words)
dictionary = corpora.Dictionary(docs)  # This creates a dictionary of all unique words

# Convert document into the bag-of-words format
corpus = [dictionary.doc2bow(doc) for doc in docs]

# Apply TF-IDF
tfidf = models.TfidfModel(corpus)  # Initialize a model
corpus_tfidf = tfidf[corpus]  # Apply transformation to the entire corpus

# Running LDA using TF-IDF
lda_model_tfidf = LdaModel(corpus_tfidf, num_topics=10, id2word=dictionary, passes=15, random_state=100)

# To view the top 10 words for each topic
for i, topic in lda_model_tfidf.print_topics(-1, num_words=10):
    print('Topic: {} \nWords: {}'.format(i, topic))


Topic: 0 
Words: 0.002*"termination" + 0.002*"refer" + 0.001*"improving" + 0.001*"transgender" + 0.001*"polyamorous" + 0.001*"goodbye" + 0.001*"seasonal" + 0.001*"terminate" + 0.001*"essentially" + 0.001*"shared"
Topic: 1 
Words: 0.003*"diagnosis" + 0.003*"consultation" + 0.002*"bother" + 0.002*"faith" + 0.002*"courage" + 0.002*"worthless" + 0.001*"expertise" + 0.001*"domestic" + 0.001*"gut" + 0.001*"increased"
Topic: 2 
Words: 0.011*"que" + 0.010*"de" + 0.008*"la" + 0.005*"en" + 0.004*"el" + 0.004*"tu" + 0.003*"e" + 0.003*"para" + 0.003*"con" + 0.003*"te"
Topic: 3 
Words: 0.002*"dog" + 0.001*"seizure" + 0.001*"depressive" + 0.001*"breakup" + 0.001*"id" + 0.001*"craving" + 0.001*"card" + 0.001*"deepest" + 0.001*"affordable" + 0.001*"psychopath"
Topic: 4 
Words: 0.002*"certainly" + 0.002*"weekend" + 0.001*"brave" + 0.001*"apprehensive" + 0.001*"evaluation" + 0.001*"cheat" + 0.001*"wedding" + 0.001*"alliance" + 0.001*"extremely" + 0.001*"military"
Topic: 5 
Words: 0.002*"helpfulness" + 0

In [51]:
# Assume lda_model_tfidf is your trained LDA model
top_words_per_topic = []

for t in range(lda_model_tfidf.num_topics):
    top_words = lda_model_tfidf.show_topic(t, 10)
    topic_words = ", ".join([word for word, prob in top_words])
    top_words_per_topic.append((f"Topic {t+1}", topic_words))

# Create a DataFrame
df_top_words = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Words'])

# Display the DataFrame
df_top_words


,Topic,Words
0,Topic 1,"termination, refer, improving, transgender, po..."
1,Topic 2,"diagnosis, consultation, bother, faith, courag..."
2,Topic 3,"que, de, la, en, el, tu, e, para, con, te"
3,Topic 4,"dog, seizure, depressive, breakup, id, craving..."
4,Topic 5,"certainly, weekend, brave, apprehensive, evalu..."
5,Topic 6,"helpfulness, nugget, four, blog, dream, bf, li..."
6,Topic 7,"nan, imposter, syndrome, dream, approval, anti..."
7,Topic 8,"obsession, no, coverage, postpartum, empty, pr..."
8,Topic 9,"dream, animal, bed, excitement, performance, s..."
9,Topic 10,"relationship, therapist, feeling, may, would, ..."
